<a href="https://colab.research.google.com/github/mille-s/GEM24_D2T_StratifiedSampling/blob/main/GEM24_D2T_utils.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1 - D2T data selection

## Prepare repo

In [ ]:
#@title Install packages

from IPython.display import clear_output
! pip install datasets
! pip install json2html

clear_output()

In [ ]:
#@title Import required libraries

import os
import xml.etree.ElementTree as ET
from sklearn.model_selection import train_test_split
import pandas as pd
import csv
import random
import json

## Set parameters

In [ ]:
#@title Filepath definition (upload file(s) in TESTDATA folder after running!)

# project_dir_path = os.path.join('/', 'content', 'drive', 'MyDrive', 'WebNLG_data_selection')
project_dir_path = '/content'
rdf_path = os.path.join(project_dir_path, 'testdata')
csv_path = os.path.join(project_dir_path, 'csv_sampling')

subtask = 'D2T-1'#@param['D2T-1', 'D2T-2']
dataset = 'CFA'#@param['CFA', 'FA', 'FI']
# seed used for GEM'24: 49
seed = 49#@param
seed = int(seed)
datacode = subtask+'-'+dataset

output_path = os.path.join(csv_path, datacode+'_samplingData.csv')

if not os.path.exists(rdf_path):
  os.makedirs(rdf_path)

if not os.path.exists(csv_path):
  os.makedirs(csv_path)

## Create csv file with sampling info. Run once for each file.

In [ ]:
#@title Function for sampling.

def extract_data(rdf_filepath, stratify_categories, exclude_size):

  '''
      This method:
      a. extracts the required entries (RDF triple(s), number of triples, property and category) from the json file.
      b. categorizes the triple and verbalisation pair as seen/unseen category based on its presence in the training set.
      c. groups the required extracted entry field (in this case, number of triples and property) for stratified selection.
  '''

  data = []
  count = 0
  original_id = 1
  for filename in os.listdir(rdf_filepath):
    if '.xml' in filename and datacode in filename:
      tree = ET.parse(f"{rdf_filepath}/{filename}")
      root = tree.getroot()

      # extract triples
      for entry in root.findall('./entries/entry'):
        triples = []
        pred = []
        for triple in entry.find('modifiedtripleset').findall('mtriple'):
          str_triple = triple.text
          triples.append(str_triple)
          only_pred = str_triple.split('|')[1]
          pred.append(only_pred)
        if exclude_size == 'none' or (exclude_size == '1 only' and int(entry.attrib['size']) > 1) or (exclude_size == '1 and 2' and int(entry.attrib['size']) > 2):
          curr_entry = {
              'id': count,
              'original_id': original_id,
              'triples': triples.copy(),
              'property': pred.copy(),
              'num_triples': int(entry.attrib['size']),
              'category': 'unseen' if entry.attrib['category'] in ['Athlete', 'Artist', 'CelestialBody', 'MeanOfTransportation', 'Politician'] else 'seen',
              'category_all': entry.attrib['category']
          }
          if stratify_categories == 'seenUnseen':
            curr_entry['strat_field'] = str(curr_entry['num_triples'])+curr_entry['category']
          elif stratify_categories == 'allCategories':
            curr_entry['strat_field'] = str(curr_entry['num_triples'])+curr_entry['category_all']
          elif stratify_categories == 'ignoreCategories':
            curr_entry['strat_field'] = str(curr_entry['num_triples'])
          data.append(curr_entry)
          count += 1
        original_id += 1

  # Remove data points for which there is only one member in a stratify category (triggers an error when stratifying, needs 2 members min)
  clean_data = []
  # Make a dico with the count of instances of each strat_field
  count_strat_field_instances = {}
  for datapoint in data:
    if datapoint['strat_field'] in count_strat_field_instances:
      count_strat_field_instances[datapoint['strat_field']] += 1
    else:
      count_strat_field_instances[datapoint['strat_field']] = 1
  # If a count of a strat_field is one, do no include it in the final dataset
  for datapoint_clean in data:
    if count_strat_field_instances[datapoint_clean['strat_field']] == 1:
      print(f"  Removed datapoint  {datapoint_clean['strat_field']} because there is only one member!")
    else:
      clean_data.append(datapoint_clean)

  return clean_data

In [ ]:
#@title Sampling parameters

stratify_categories = 'ignoreCategories'#@param['allCategories', 'seenUnseen', 'ignoreCategories']
number_samples = "180"#@param[50, 100, 120, 150, 175, 180, 200, 300, 400, 500]
num_samples = int(number_samples)
exclude_size = '1 only'#@param['none', '1 only', '1 and 2']
# Get data
data=extract_data(rdf_path, stratify_categories, exclude_size)

In [ ]:
#@title Stratified selection using train_test_split

# tset = pd.DataFrame.from_dict(data)

# X_train, X_test, = train_test_split(tset, test_size=num_samples, random_state=seed, stratify=tset['strat_field'], shuffle=True)
# print(len(X_train), len(X_test))


In [ ]:
#@title Balanced selection using groupby

tset = pd.DataFrame.from_dict(data)

# I found three ways, not sure what they exactly do; the second and third ones allow for specifying the random_state
# X_test = tset.groupby(tset['strat_field']).apply(lambda s: s.sample(30))
# X_test = tset.sample(frac = 1.0, random_state=seed, axis=0).groupby(tset['strat_field']).head(30)
# The third one below seems more controlled; grouby uses axis=0
X_test = tset.groupby(by=tset['strat_field']).sample(n = 30, random_state=seed)

print(X_test)

In [ ]:
#@title Print some numbers
# tset['num_triples']
# len(tset.loc[tset['category'] == 'unseen'])
# print(X_test['num_triples'])

# Show mean of column that contains triple number in each input (https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.loc.html)
# print(X_test.loc[:, 'num_triples'].mean())
print(f"{round(X_test['num_triples'].mean(), 2)} triples per input on average")

def count_num_instances(pd_column):
  count = {}
  for category in pd_column:
    if category in count:
      count[category] += 1
    else:
      count[category] = 1

  for count_category in sorted(count):
    print(f'{count_category}\t{count[count_category]}')
    # print(f'{count[count_category]}')
  print('-----------------')

count_num_instances(X_test['num_triples'])
count_num_instances(X_test['category_all'])

In [ ]:
#@title Play around with groupby
# df_test_gb = pd.DataFrame({'Animal': ['Falcon', 'Falcon',
#                               'Parrot', 'Parrot'],
#                    'Max Speed': [380., 370., 24., 26.]})
# print(df_test_gb)

# print(df_test_gb.groupby(['Animal']).mean())

In [ ]:
#@title Create CSV file
X_test.to_csv(output_path, index=False)

# 2 - Create HTML tables for inputs. Run once for each file.

In [ ]:
#@title Generate HTML tables from pandas dataframe (running the code above before)
from json2html import *
import json
import codecs
import os

path_out_pd = '/content/tables_pd'
if not os.path.exists(path_out_pd):
  os.makedirs(path_out_pd)

for ind in X_test.index:
  triple_set = X_test['triples'][ind]
  orig_id = X_test['original_id'][ind]
  # orig_id starts numbering at 1, while the lists of outputs texts will start numbering at 0, so we need to remove 1 from the original ID to maintain alignment with output files lines
  with codecs.open(os.path.join(path_out_pd, f'{datacode}_{str(orig_id-1).rjust(4, "0")}.html'), 'w', 'utf-8') as fo:
    list_dico_input = []
    for triple in triple_set:
      dico_triples = {}
      dico_triples['Subject'] = triple.split(' | ')[0]
      dico_triples['Property'] = triple.split(' | ')[1]
      dico_triples['Object'] = triple.split(' | ')[2]
      list_dico_input.append(dico_triples)
    fo.write(json2html.convert(json = list_dico_input))

In [ ]:
#@title Generate HTML tables from HuggingFace data

# from datasets import load_dataset
# from json2html import *
# import codecs
# import os
# import re
# import json

# # struct2text: common_gen, cs_restaurants, dart, e2e_nlg, totto, web_nlg_en, web_nlg_ru
# # schema_guided_dialog
# dataset_name = 'web_nlg_en'
# dataset = load_dataset('gem', dataset_name)
# # subSets = ['test', 'validation', 'train']
# subSets = ['test']

# for subSet in subSets:
#   x = 0
#   subSet_inputs = dataset[subSet]
#   while x < len (subSet_inputs):
#     # fileName_out = 'out_tables/'+dataset_name+'-'+subSet+'-'+str(x)+'.html'
#     fileName_out = 'tables_/'+subSet_inputs[x]['gem_id']+'.html'
#     if not os.path.exists('tables_'):
#       os.makedirs('tables_')
#     print('Processing '+fileName_out)
#     fo = codecs.open(fileName_out, 'w', 'utf-8')
#     list_dico_input = []
#     if dataset_name == 'web_nlg_en' or dataset_name == 'web_nlg_ru':
#       for triple in subSet_inputs[x]['input']:
#         dico_triples = {}
#         dico_triples['Subject'] = triple.split(' | ')[0]
#         dico_triples['Property'] = triple.split(' | ')[1]
#         dico_triples['Object'] = triple.split(' | ')[2]
#         list_dico_input.append(dico_triples)
#     elif dataset_name == 'common_gen':
#       dico_concepts = {}
#       dico_concepts['Concepts'] = subSet_inputs[x]['concepts']
#       list_dico_input.append(dico_concepts)
#     elif dataset_name == 'cs_restaurants':
#       dico_DAs = {}
#       DA = subSet_inputs[x]['dialog_act'].split('(')[0]
#       triples = subSet_inputs[x]['dialog_act'].split('(')[1].split(')')[0]
#       dico_DAs['Dialogue Act'] = DA
#       if re.search(',', triples):
#         dico_DAs['Topics'] = triples.split(',')
#       else:
#         dico_DAs['Topic'] = triples
#       list_dico_input.append(dico_DAs)
#     elif dataset_name == 'e2e_nlg':
#       list_properties = subSet_inputs[x]['meaning_representation'].split(', ')
#       for input_property in list_properties:
#         dico_properties = {}
#         prop_name = input_property.split('[')[0]
#         prop_value = input_property.split('[')[1].split(']')[0]
#         dico_properties['Property'] = prop_name
#         dico_properties['Value'] = prop_value
#         list_dico_input.append(dico_properties)
#     elif dataset_name == 'schema_guided_dialog':
#       dico_DAs = {}
#       dico_DAs['Dialogue Acts'] = subSet_inputs[x]['dialog_acts']
#       list_dico_input.append(dico_DAs)
#     fo.write(json2html.convert(json = list_dico_input))
#     fo.close()
#     x += 1

## Download files

In [ ]:
#@title Zip and download tables
from google.colab import files
zip_name_inter = '/content/html_tables.zip'
!zip -r {zip_name_inter} /content/tables_pd

clear_output()

files.download(zip_name_inter)

In [ ]:
#@title Zip and download CSVs
from google.colab import files
zip_name_inter = '/content/CSVs.zip'
!zip -r {zip_name_inter} /content/csv_sampling

clear_output()

files.download(zip_name_inter)

In [ ]:
#@title Check the contents of the created CSVs
import glob
import pandas as pd
import os

csv_files = glob.glob(os.path.join(csv_path, '*.csv'))

dico_properties = {}

for csv_file in sorted(csv_files):
  props_unique = []
  count_props_all = 0
  head, tail = os.path.split(csv_file)
  # print(tail)
  df_triple_sets = pd.read_csv(csv_file)['triples']
  # print(df_triple_sets)
  for triple_set in df_triple_sets:
    triples_list = triple_set.replace('"', "'").replace("['", "").replace("']", "").split("', '")
    properties_list = [triple.split(' | ')[1] for triple in triples_list]
    for triple_property in properties_list:
      count_props_all += 1
      if triple_property not in props_unique:
        props_unique.append(triple_property)
  dico_properties[tail] = props_unique
  # print(f'  {count_props_all} properties found (810 expected).')

for dataset in dico_properties:
  print(dataset, ':', len(dico_properties[dataset]), 'different properties found:')
  print('  ', sorted(dico_properties[dataset]))

# 3 - Create individual files for sampled system outputs

First upload all system outputs in a folder named "sys_outputs", and generate the corresponding csv file(s) with the code above (or upload manually in a folder called "csv_sampling").

In [ ]:
#@title Create one text file per sampled input/output per team per language per test set
import os
import glob
from pandas import *
import codecs

out_sampled_folder = '/content/d2t_outputs-sampled'

# The 3 CSV files for each task have the same IDs sampled, so we can just use one file per task
list_csv_D2T1_OIDs = read_csv('/content/csv_sampling/D2T-1-FA_samplingData.csv')['original_id'].tolist()
list_csv_D2T2_OIDs = read_csv('/content/csv_sampling/D2T-2-FA_samplingData.csv')['original_id'].tolist()

# The original IDs are numbered starting from 1, we want a number starting from 0 to aling with list indices in the system output files; make it into a dic for easy access afterwards
list_csv_ids = {}
list_csv_ids['D2T-1'] = [OID1-1 for OID1 in sorted(list_csv_D2T1_OIDs)]
list_csv_ids['D2T-2'] = [OID2-1 for OID2 in sorted(list_csv_D2T2_OIDs)]

for sys_output_path in glob.glob(os.path.join('/content/sys_outputs', '*.txt')):
  head, tail = os.path.split(sys_output_path)
  # Get parameters of every output file
  team_ID = tail.split('-', 1)[0]
  lang_out = tail.rsplit('.', 1)[0].rsplit('_', 1)[1]
  data_code_out = tail.rsplit('.', 1)[0].rsplit('_', 1)[0].rsplit('_', 1)[1]
  subtask_code_out = data_code_out.rsplit('-', 1)[0]
  # print(tail)
  # print(f'  {team_ID}')
  # print(f'  {lang_out}')
  # print(f'  {data_code_out}')
  # print(f'  {subtask_code_out}')

  # Create subfolder to store sampled system outputs
  dest_folder_sample = os.path.join(out_sampled_folder, data_code_out, lang_out, team_ID)
  if not os.path.exists(dest_folder_sample):
    os.makedirs(dest_folder_sample)

  # Read sys output
  sys_output_all_lines = codecs.open(sys_output_path).readlines()

  for id_sampled in list_csv_ids[subtask_code_out]:
    # Create text files the last part of the name of which matches the name of the sampled input files
    dest_filename_sample = os.path.join(dest_folder_sample, '['+team_ID+'_'+lang_out+']_'+data_code_out+'_'+str(id_sampled).rjust(4, "0")+'.txt')
    with codecs.open(dest_filename_sample, 'w', 'utf-8') as fo:
        fo.write(sys_output_all_lines[id_sampled].strip())


In [ ]:
#@title Zip and download sampled output text files
from IPython.display import clear_output
from google.colab import files
zip_name_inter = '/content/d2t_outputs-sampled.zip'
!zip -r {zip_name_inter} /content/d2t_outputs-sampled
clear_output()

files.download(zip_name_inter)

# 4 - Create individual files for reference texts collected on AMT

In [ ]:
#@title Read json and check texts (UPLOAD FIRST)
import json
import re
import codecs
import os
import glob

# First upload '/content/en_references_v2-formatted.json' and '/content/csv_sampling/d2t_outputs-sampled-onlyWebnlgRef_grouped_checked.zip'

# Load json
references_json = json.load(open('/content/en_references_v2-formatted.json'))
# Unzip sampled output folder, which contains lists of IDs of sampled outputs
if not os.path.exists('/content/content/d2t_outputs-sampled_grouped'):
  ! unzip '/content/csv_sampling/d2t_outputs-sampled-onlyWebnlgRef_grouped_checked.zip'

# Get list of IDs for all sampled outputs
list_IDs_sampled = []
for dataset_path in glob.glob(os.path.join('/content/content/d2t_outputs-sampled_grouped', '*')):
  head, tail = os.path.split(dataset_path)
  # print(tail)
  path_IDs = os.path.join(dataset_path, 'en', 'AllSizes', 'AllSizes_sampled_ids.txt')
  # Open IDs file, read, and split the list on the first line
  datasets_IDs_sampled = codecs.open(path_IDs, 'r', 'utf-8').readlines()[0].strip().replace('[', '').replace(']', '').replace("'", '').split(', ')
  for dataset_ID_sampled in datasets_IDs_sampled:
    full_ID = f'en_{tail}_{dataset_ID_sampled}'
    list_IDs_sampled.append(full_ID)
# print(len(list_IDs_sampled))
# print(list_IDs_sampled)

ids_error = []
all_ids_collected = []
# Find some errors
with codecs.open('/content/writing_problems.txt', 'w', 'utf-8') as fo1:
  for reference_text in references_json["references"]:
    id = reference_text["short_id"]
    text = reference_text["english_output"].strip().replace('\r\n', ' ')
    all_ids_collected.append(id)
    # Capture people who pasted some definitions
    if re.search('\[', text):
      fo1.write(f'{id} (pasted definition): {text}\n')
      print(f'{id} (pasted definition): {text}')
      ids_error.append(id)
    # Capture people who pasted the input
    elif re.search('Subject', text):
      fo1.write(f'{id} (pasted input): {text}\n')
      print(f'{id} (pasted input): {text}')
      ids_error.append(id)

  # Find missing files
  for ID_sampled in list_IDs_sampled:
    if ID_sampled not in all_ids_collected:
      fo1.write(f'{ID_sampled} (missing)\n')
      print(f'{ID_sampled} (missing)')

if len(ids_error) == 0:
  print('No errors found!')

ids_seen = []
texts_seen = []
with codecs.open('/content/writing_duplicates.txt', 'w', 'utf-8') as fo2:
  for reference_text in references_json["references"]:
    id = reference_text["short_id"]
    text = reference_text["english_output"].strip().replace('\r\n', ' ')
    # Check if there is a duplicate for a text with error
    if id in ids_error:
      print(f'Found alternative to error {id}')
    # Capture duplicate outputs
    if id not in ids_seen:
      ids_seen.append(id)
      texts_seen.append(text)
    else:
      index_id = ids_seen.index(id)
      fo2.write(f'{id} (duplicate): {texts_seen[index_id]}\n')
      # print(f'{id} (duplicate): {texts_seen[index_id]}')



In [ ]:
#@title Create files
import json
import os
import codecs

# First upload '/content/en_references-formatted.json'

# Create folders for text files
path_collected_references = os.path.join('/content', 'texts_AMT')
if not os.path.exists(path_collected_references):
  os.makedirs(path_collected_references)

# Load json
ids_seen = []
references_json = json.load(open('/content/en_references_v2-formatted.json'))
for reference_text in references_json["references"]:
  id = reference_text["short_id"]
  # We only take the first text available
  if id not in ids_seen:
    ids_seen.append(id)
    language = id.split('_', 1)[0]
    id_nolang = id.split('_', 1)[1]
    text = reference_text["english_output"].strip().replace('\r\n', ' ')
    filename = os.path.join(path_collected_references, f'[0_{language}]_{id_nolang}.txt')
    with codecs.open(filename, 'w', 'utf-8') as fo:
      fo.write(text)
print(f'{str(len(ids_seen))} files were created (1,080 expected.)')

In [ ]:
#@title Zip and download texts
from google.colab import files
from IPython.display import clear_output

zip_name_inter = '/content/texts_AMT.zip'
!zip -r {zip_name_inter} /content/texts_AMT

clear_output()

files.download(zip_name_inter)

# 5 - Create files for automatic evaluation by size (sizes 2 to 7 and all sizes together)

In [ ]:
#@title Group references and system outputs by size
from posix import write
import glob
import pandas as pd
import os
import re
import codecs
import shutil

language_out = 'en' #['en', 'es', 'sw']
teams_withdrawn = ['8']
# First upload '/content/csv_sampling/d2t_inputs-sampled.zip' and /content/csv_sampling/d2t_outputs-sampled.zip

folder_suffix = '-enAmtRefs' #param['-onlyWebnlgRef', '-enAmtRefs']
outputs_sampled_folder= '/content/csv_sampling/d2t_outputs-sampled'+folder_suffix+'.zip'
inputs_sampled_folder= '/content/csv_sampling/d2t_inputs-sampled.zip'

# Unzip sampled input and output files if they have not been unzipped already:
if not os.path.exists('/content/d2t_inputs-sampled'):
  ! unzip {outputs_sampled_folder}
  ! unzip {inputs_sampled_folder}

# Delete /content/d2t_outputs-sampled_grouped folder
if os.path.exists('/content/d2t_outputs-sampled'+folder_suffix+'_grouped'):
  shutil.rmtree('/content/d2t_outputs-sampled'+folder_suffix+'_grouped')

def writeFile(file_to_write, content, total_num_lines, current_num_lines):
    file_to_write.write(content)
    # After all lines except the last one, insert linebreak
    if current_num_lines < total_num_lines-1:
      file_to_write.write('\n')

# Create new output folders for storing files with grouped outputs for each team
subfolders_out = sorted([os.path.join(f.path, language_out) for f in os.scandir('/content/d2t_outputs-sampled'+folder_suffix+'/d2t_outputs-sampled') if f.is_dir()])
# print(subfolders_out)
subfolders_out_grouped = []
for subfolder_out in subfolders_out:
  # '/content/d2t_outputs-sampled-onlyWebnlgRef/d2t_outputs-sampled/D2T-1-CFA/en'
  new_folder_name = re.sub('d2t_outputs-sampled'+folder_suffix, 'd2t_outputs-sampled'+folder_suffix+'_grouped', subfolder_out)
  subfolders_out_grouped.append(new_folder_name)
  if not os.path.exists(new_folder_name):
    os.makedirs(new_folder_name)

# Read files with sampling info (we only need one file per subtask since all IDs are the same for each subtask)
csv_files = glob.glob(os.path.join('/content/d2t_inputs-sampled/info_sampling-csv', '*-FA_samplingData.csv'))

# Build a dico with the IDs grouped by size
# {'D2T-1': {2: [46, 920, 936, 1423, 1723, 410, 1653, 1589, 762, 1471, 590, 1685, 1217, 1355, 99, 1339, 1079, 1420, 276, 771, 757, 845, 472, 1222, 1072, 582, 1043, 360, 1387, 1113], ...}}
dico_ids_per_size = {}
for csv_file in sorted(csv_files):
  head, tail = os.path.split(csv_file)
  task = tail.rsplit('-', 1)[0]
  df_triple_sets = pd.read_csv(csv_file)
  dico_ids_per_size[task] = {}
  for n in range(2, 8):
    dico_ids_per_size[task][n] = [df_triple_sets.loc[df_triple_sets['num_triples'] == n, 'original_id'].tolist()][0]
# print(dico_ids_per_size)
# print(subfolders_out)
# print(subfolders_out_grouped)

# subfolder_out is something like /content/d2t_outputs-sampled/D2T-1-CFA/en
for subfolder_out, subfolder_out_grouped in zip(subfolders_out, subfolders_out_grouped):
  list_sampled_IDs = []
  # Get folders inside subfolder_out
  teams_out_folders = sorted([f.path for f in os.scandir(subfolder_out) if f.is_dir() and f.name not in teams_withdrawn])
  # print(teams_out_folders)
  # For each team out folder, check if the path contains one of the keys in dico_ids_per_size
  # team_out_folder is something like /content/d2t_outputs-sampled/D2T-1-CFA/en/1
  for team_out_folder in teams_out_folders:
    num_of_IDs_total = 0
    head_tf, team_id = os.path.split(team_out_folder)
    print(team_out_folder)
    # There are 2 subtasks in dico_ids_per_size: D2T-1 and D2T-2
    for task in dico_ids_per_size.keys():
      # Make sure we are in the output folder of the right subtask
      if task in team_out_folder:
        # print(f'  {task} found in {team_out_folder}')
        # Within each subtask, we want to create separate files for each input size
        for size in dico_ids_per_size[task]:
          # print(f'  NumOfIDs: {num_of_IDs_total}')
          print(f'    {size}: {dico_ids_per_size[task][size]}')
          # Create a folder for each size (to run the eval later on with my notebook, I need to process all hypotheses for one (set of) reference file(s)).
          if not os.path.exists(os.path.join(subfolder_out_grouped, 'Size'+str(size))):
            os.makedirs(os.path.join(subfolder_out_grouped, 'Size'+str(size)))
          # Create a folder where we'll group all the sizes for each system in one file
          if not os.path.exists(os.path.join(subfolder_out_grouped, 'AllSizes')):
            os.makedirs(os.path.join(subfolder_out_grouped, 'AllSizes'))

          # Get a padded version of all IDs in dico_ids_per_size[task][size]
          for num_of_IDs_for_a_size, id in enumerate(dico_ids_per_size[task][size]):
            # We need to subtract 1 to the original ID to match the ID of the files, for which the numbering starts from 0
            padded_aligned_id = str(id-1).zfill(4)
            # Now iterate over all the output files of the current folder
            for team_out_file_path in glob.glob(os.path.join(team_out_folder, '*.txt')):
              head_tf, tail_tf = os.path.split(team_out_file_path)
              # Look for files that match one of the IDs of texts of a specific size
              if padded_aligned_id in tail_tf:
                with codecs.open(team_out_file_path, 'r', 'utf-8') as ftext:
                  contents_read = ftext.read()
                  # Write separate files for all sizes
                  with codecs.open(os.path.join(subfolder_out_grouped, 'Size'+str(size), str(team_id)+'_Size'+str(size)+'.txt'), 'a', 'utf-8') as f_sep:
                    writeFile(f_sep, contents_read, len(dico_ids_per_size[task][size]), num_of_IDs_for_a_size)
                  # Write one file with all sizes grouped
                  with codecs.open(os.path.join(subfolder_out_grouped, 'AllSizes', str(team_id)+'_AllSizes.txt'), 'a', 'utf-8') as f_all:
                    writeFile(f_all, contents_read, len(dico_ids_per_size[task][size])*len(dico_ids_per_size[task]), num_of_IDs_total)
            num_of_IDs_total += 1

          # Now gather the IDs of the inputs that will match the filenames for each dataset and size
          padded_aligned_id_list = [str(id-1).zfill(4) for id in dico_ids_per_size[task][size]]
          with codecs.open(os.path.join(subfolder_out_grouped, 'Size'+str(size), 'Size'+str(size)+'_sampled_ids.txt'), 'w', 'utf-8') as f_id_sep:
            f_id_sep.write(str(sorted(padded_aligned_id_list)))
          # Add all the IDs into a list for all sizes
          for padded_aligned_id2 in padded_aligned_id_list:
            if padded_aligned_id2 not in list_sampled_IDs:
              list_sampled_IDs.append(padded_aligned_id2)
  with codecs.open(os.path.join(subfolder_out_grouped, 'AllSizes', 'AllSizes_sampled_ids.txt'), 'a', 'utf-8') as f_id_all:
    f_id_all.write(str(sorted(list_sampled_IDs)))




In [ ]:
#@title Zip and download files
from google.colab import files
from IPython.display import clear_output

folder_to_zip = '/content/d2t_outputs-sampled'+folder_suffix+'_grouped'
zip_name_inter = folder_to_zip+'.zip'
!zip -r {zip_name_inter} {folder_to_zip}

clear_output()

files.download(zip_name_inter)

# 6 - Create plots from automatic evaluation logs

In [ ]:
#@title Parse log files and create plots per size
import os
import glob
import matplotlib.pyplot as plt
import numpy as np
import re
import shutil

# First upload the log_eval.zip file (for 1 dataset) obtained with this notebook: https://github.com/mille-s/WebNLG-2020_Metrics

dataset = 'D2T-1-FA' #@param['D2T-1-FA', 'D2T-1-CFA', 'D2T-1-FI', 'D2T-2-FA', 'D2T-2-CFA', 'D2T-2-FI']
# Unzip log_eval folder
# For when we zip the folders inside the content folder
path_logEval = '/content/content/log_eval'
path_zip = '/content/log_eval_'+dataset+'.zip'
if os.path.exists(path_logEval):
  shutil.rmtree(path_logEval)
  ! unzip {path_zip}
else:
  ! unzip {path_zip}

# dataset = 'D2T-1-FA'#@param['D2T-1-CFA', 'D2T-1-FA', 'D2T-1-FI', 'D2T-2-CFA', 'D2T-2-FA', 'D2T-2-FI']
metrics = ['BLEU', 'METEOR', 'chrF++', 'BERT-SCORE F1']
system_IDnames_dico = {'1':'DCU-ADAPT-modPB', '2':'DCU-NLG-PBN', '3':'DCU-NLG-Small', '4':'DipInfo-UniTo', '5':'OSU-CompLing', '6':'RDFpyrealb', '7':'SaarLST'}

# dico_plot_BLEU = {}
# dico_plot_METEOR = {}
# dico_plot_chrF = {}
# dico_plot_BERT = {}
dico_plot_per_size = {}
dico_plot_all_sizes = {}
eval_files_paths = glob.glob(os.path.join(path_logEval, '*.txt'))
for eval_file_path in sorted(eval_files_paths):
  head, tail = os.path.split(eval_file_path)
  # values for "size": 2 to 7, and "s" for all sizes
  size = tail.rsplit('_', 1)[1].rsplit('.', 1)[0].split('Size')[1]
  sys_id = tail.rsplit('_', 1)[0].rsplit('_', 1)[1]
  dataset = tail.rsplit('_', 3)[0].rsplit('_', 1)[1]
  # print(f'{sys_id}  {size}')

  # Make a key in dico_plot with a new system id
  # if sys_id not in dico_plot_BLEU:
  #   dico_plot_BLEU[sys_id] = []
  # if sys_id not in dico_plot_METEOR:
  #   dico_plot_METEOR[sys_id] = []
  # if sys_id not in dico_plot_chrF:
  #   dico_plot_chrF[sys_id] = []
  # if sys_id not in dico_plot_BERT:
  #   dico_plot_BERT[sys_id] = []

  # The scores are on the penultimate line of the log file, the metrics names 2 lines above
  log_lines_metrics = [line.rstrip('\n') for line in open(eval_file_path)][-3]
  log_lines_scores = [line.rstrip('\n') for line in open(eval_file_path)][-1]
  # Split lines with names and scores
  # Metrics are separated by 4 spaces, with 2 initial spaces; some metrics have a space inside the name
  metrics_list = [x for x in log_lines_metrics.split('  ') if not x == '']
  # Scores are separated by an unknown number of spaces that depends on the metric name (to show metrics and scores aligned)
  scores_list = [x for x in log_lines_scores.split(' ') if not x == '']
  # Get the pairs metric/score for the metrics we are interested in
  for metric_name, score in zip(metrics_list, scores_list):
    if metric_name in metrics:
      # print(f'{metric_name}: {score}')
      if not size == 's':
        if metric_name not in dico_plot_per_size:
          dico_plot_per_size[metric_name] = {}
        if sys_id not in dico_plot_per_size[metric_name]:
          dico_plot_per_size[metric_name][sys_id] = []
        dico_plot_per_size[metric_name][sys_id].append(float(score))
      else:
        # print(f'{sys_id} AllSizes')
        # print(f'{metric_name}: {score}')
        if sys_id not in dico_plot_all_sizes:
          dico_plot_all_sizes[sys_id] = []
        dico_plot_all_sizes[sys_id].append(float(score))
  # print(metrics_list)
  # print(scores_list)


# Print the lines to put in a latex table
print(f'\nResults all sizes (order: BLEU, METEOR, chrF++, BERT-SCORE F1):')
print(dico_plot_all_sizes)
for sys_id in dico_plot_all_sizes:
  print(system_IDnames_dico[sys_id]+' & '+' & '.join([str(x) for x in dico_plot_all_sizes[sys_id]])+' \\\\')

print(f'\nResults per size:')
print(dico_plot_per_size)

system_colors = {'1':'dodgerblue', '2':'blue', '3':'orange', '4':'green', '5':'red', '6':'purple', '7':'brown'}
# Make and save plots in png
for metric_name in dico_plot_per_size:
  for sys_id in dico_plot_per_size[metric_name]:
    plt.plot(range(2, len(dico_plot_per_size[metric_name][sys_id]) + 2), dico_plot_per_size[metric_name][sys_id], label=system_IDnames_dico[sys_id], color=system_colors[sys_id])
  plt.title(f'{dataset}: {metric_name}')
  plt.xlabel("Input size")
  plt.ylabel("Score")
  plt.legend()
  #https://blog.timodenk.com/exporting-matplotlib-plots-to-latex/
  # I can't generate the plots inside latex
  # plt.savefig(f'{dataset}_{metric_name}.pgf')
  # Save the plot as a png file
  plt.savefig(f'{dataset}_{metric_name}.png')
  # plt.show needs to be after savefig, otherwise the image is blank
  plt.show()



In [ ]:
#@title Parse log files and create plots for all sizes

import matplotlib.pyplot as plt

# Metrics order in dico_plot_all_sizes: BLEU, METEOR, chrF++, BERT-SCORE F1
metrics_order = ['BLEU', 'METEOR', 'chrF++', 'BERT']
metrics_lower_limit = [0, 0.2, 0.4, 0.9]
metrics_upper_limit = [40, 0.4, 0.6, 0.95]
system_IDShortnames_dico = {'2':'DCU-PBN', '3':'DCU-Sm', '4':'DipInfo', '5':'OSU', '6':'pyrealb', '7':'Saar'}
teams = [system_IDShortnames_dico[x] for x in dico_plot_all_sizes.keys() if not x == '1']

for metric_id in range(4):
  fig, ax = plt.subplots()
  bar_labels = teams
  bar_colors = ['tab:blue', 'tab:orange', 'tab:green', 'tab:red', 'tab:purple', 'tab:brown']
  scores = [dico_plot_all_sizes[x][metric_id] for x in dico_plot_all_sizes.keys() if not x == '1']
  ax.bar(teams, scores, label=bar_labels, color=bar_colors)
  plt.ylim(metrics_lower_limit[metric_id], metrics_upper_limit[metric_id])
  ax.set_ylabel('Score')
  ax.set_title(metrics_order[metric_id]+'-'+dataset)
  # ax.legend(title='Teams')
  plt.savefig(f'{dataset}_{metrics_order[metric_id]}_allSys.png')
  plt.show()

  # Also create a Latex table for the paper



In [ ]:
#@title Calculate D2T score differences for each system on the different datasets

system_IDShortnames = ['DCU-PBN', 'DCU-Sm', 'DipInfo', 'OSU', 'pyrealb', 'Saar']
#BLEU (Systems 2 to 7): one list of 6 scores per system (scores: 1-FA, 1-CFA, 1-FI, 2-FA, 2-CFA, 2-FI)
bleuScores = [[29.08, 25.2, 26.02, 23.96, 30.34, 20.46], [27.0, 22.98, 20.85, 19.48, 24.9, 16.88], [32.31, 29.01, 28.24, 27.22, 32.01, 21.26], [30.03, 24.45, 21.44, 24.97, 27.06, 16.9], [26.37, 21.67, 21.97, 19.97, 25.05, 16.28], [29.7, 23.48, 20.76, 28.25, 26.47, 20.16]]
#METEOR (Systems 2 to 7): one list of 6 scores per system (scores: 1-FA, 1-CFA, 1-FI, 2-FA, 2-CFA, 2-FI)
meteorScores = [[0.33, 0.297, 0.322, 0.295, 0.348, 0.3], [0.314, 0.279, 0.292, 0.26, 0.3, 0.267], [0.346, 0.315, 0.342, 0.304, 0.354, 0.307], [0.335, 0.293, 0.306, 0.295, 0.334, 0.282], [0.331, 0.291, 0.31, 0.287, 0.335, 0.286], [0.347, 0.307, 0.331, 0.32, 0.359, 0.315]]
#CHRF (Systems 2 to 7): one list of 6 scores per system (scores: 1-FA, 1-CFA, 1-FI, 2-FA, 2-CFA, 2-FI)
chrfScores = [[0.555, 0.513, 0.549, 0.49, 0.581, 0.49], [0.537, 0.488, 0.507, 0.438, 0.51, 0.442], [0.58, 0.543, 0.587, 0.512, 0.592, 0.502], [0.566, 0.514, 0.537, 0.496, 0.567, 0.475], [0.551, 0.495, 0.527, 0.479, 0.561, 0.472], [0.581, 0.524, 0.557, 0.538, 0.597, 0.518]]
#BERT (Systems 2 to 7): one list of 6 scores per system (scores: 1-FA, 1-CFA, 1-FI, 2-FA, 2-CFA, 2-FI)
bertScores = [[0.933, 0.923, 0.92, 0.936, 0.937, 0.924], [0.93, 0.918, 0.914, 0.925, 0.923, 0.914], [0.933, 0.926, 0.924, 0.937, 0.936, 0.923], [0.932, 0.92, 0.915, 0.934, 0.93, 0.917], [0.928, 0.918, 0.917, 0.921, 0.923, 0.916], [0.931, 0.921, 0.917, 0.934, 0.929, 0.919]]

def calculateDiffs(list_scores, system_IDShortnames, metric):
  print(metric)
  for pos, systemScores in enumerate(list_scores):
    system_name = system_IDShortnames[pos]
    if system_name == 'DCU-Sm':
      print(f'  {system_name}')
      print(f'    1 FA to CFA: {round(systemScores[1]-systemScores[0], 2)}')
      print(f'    1 FA to FI: {round(systemScores[2]-systemScores[0], 2)}')
      print(f'    ------')
      print(f'    2 FA to CFA: {round(systemScores[4]-systemScores[3], 2)}')
      print(f'    2 FA to FI: {round(systemScores[5]-systemScores[3], 2)}')
      print(f'    ------')
      print(f'    FA 1 to 2: {round(systemScores[3]-systemScores[0], 2)}')
      print(f'    CFA 1 to 2: {round(systemScores[4]-systemScores[1], 2)}')
      print(f'    FI 1 to 2: {round(systemScores[5]-systemScores[2], 2)}')

calculateDiffs(bleuScores, system_IDShortnames, 'BLEU')
calculateDiffs(meteorScores, system_IDShortnames, 'METEOR')
calculateDiffs(chrfScores, system_IDShortnames, 'chrF++')
calculateDiffs(bertScores, system_IDShortnames, 'BERT')